# RAG leveraging LangChain, OpenAI, and FAISS.

In [35]:
# Install required packages (Note: You can either the below line or install packages via requirements.txt)

!pip install pypdf2 langchain faiss-cpu openai tiktoken langchain-openai

In [40]:
# Import required libraries

import faiss
from PyPDF2 import PdfReader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

In [41]:
# Declare your OPEN AI api key

OPEN_API_KEY = "<Your OPEN AI key>"

In [ ]:
# Read the input file

file = open("resources/Approved IP Law.pdf", "rb")
pdf_reader = PdfReader(file)

document_text = ""
for page in pdf_reader.pages:
  document_text += page.extract_text()


In [24]:
# Break content into chunks

text_splitter = RecursiveCharacterTextSplitter(
    separators="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

chunks = text_splitter.split_text(document_text)


In [25]:
# Instantiate OpenAIEmbeddings to generate embedding of the chunks

embeddings = OpenAIEmbeddings(openai_api_key=OPEN_API_KEY)

In [ ]:
# Establish FAISS vector store

vector_store = FAISS.from_texts(chunks, embeddings)
print(vector_store.index.ntotal)

In [43]:
# Query your vector store

query = "Who is eligible to file a patent?"
match_chunks = vector_store.similarity_search(query)

In [ ]:
# Instantiate Open AI LLM model

llm = ChatOpenAI(
  openai_api_key = OPEN_API_KEY,
  temperature = 0,
  max_tokens = 1000,
  model_name = "gpt-3.5-turbo"
)

In [45]:
# Setup the chain and run it

chain = load_qa_chain(llm, chain_type="stuff")
response = chain.run(input_documents=match_chunks, question = query)

print("Response:", response)

Response: Individuals who have come up with a novel invention resulting from an innovative idea or improvement that involves an inventive step and has industrial application are eligible to file a patent. Additionally, foreigners can also file for patents in the State if they are citizens of a country that reciprocates similar treatment with the State.
